## Initialization

In [1]:
# --- Kaggle Bootstrap Script ---
%cd /kaggle/working
%rm -rf MedCLIP
!git clone --depth 1 https://github.com/lamlethanh777/MedCLIP.git

%pwd
%cd MedCLIP

# Install dependencies
!pip install -r requirements.txt --quiet

# # Show current commit for reproducibility
!git rev-parse HEAD

/kaggle/working
Cloning into 'MedCLIP'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 39 (delta 3), reused 23 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 109.56 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/kaggle/working/MedCLIP
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

## Pulling new changes from local

In [2]:
!git pull

Already up to date.


## Convert Open-I dataset to compatible form

In [3]:
!python preprocess_indiana.py --quiet

Loading Indiana dataset...
Loaded 7466 projections
Loaded 3851 reports
Filtered to 3818 frontal views
Merging projections and reports...
Merged dataset size: 3818
Found 3818 images that exist on disk
After filtering reports: 3794 samples
Extracting labels...
Splitting data with validation ratio: 0.2
Train set: 3036 samples
Validation set: 758 samples

Training Set Label Distribution:
--------------------------------------------------
No Finding                    : 2252 (74.18%)
Enlarged Cardiomediastinum    :    0 ( 0.00%)
Cardiomegaly                  :  263 ( 8.66%)
Lung Lesion                   :    0 ( 0.00%)
Lung Opacity                  :  351 (11.56%)
Edema                         :   35 ( 1.15%)
Consolidation                 :   23 ( 0.76%)
Pneumonia                     :   33 ( 1.09%)
Atelectasis                   :  257 ( 8.47%)
Pneumothorax                  :   26 ( 0.86%)
Pleural Effusion              :  117 ( 3.85%)
Pleural Other                 :    0 ( 0.00%)
Fracture  

## Training MedCLIP with Open-I

In [4]:
!python examples/run_indiana_pretrain.py

2025-10-17 15:58:25.218198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760716705.392984      93 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760716705.441804      93 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using device: cuda:0
Training configuration:
  batch_size: 32
  num_epochs: 20
  warmup: 0.1
  lr: 2e-05
  weight_decay: 0.0001
  eval_batch_size: 64
  eval_steps: 500
  save_steps: 500

Datasets to use: ['indiana-train']

Loading training data...
load data from ./local_data/indiana-train-meta.csv
load sentence prompts from ./local_data/sentence-label.csv
build prompt sentences.
Training samples: 3036
/usr/local/lib/python3.11/dist-p

## Prompt Classification Demo

### Import required modules

In [ ]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT
from medclip import MedCLIPProcessor
from medclip import PromptClassifier

### Initialize models

In [ ]:
# init models
processor = MedCLIPProcessor()
model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT, checkpoint='./data/MedCLIP/checkpoints/vision_text_pretrain/25000')
clf = PromptClassifier(model, ensemble=True)
clf.cuda()

### Prepare input image and prompts

In [ ]:
# prepare input image
from PIL import Image
image = Image.open('./example_data/view1_frontal.jpg')
inputs = processor(images=image, return_tensors="pt")

# prepare input prompt texts
from medclip.prompts import generate_chexpert_class_prompts, process_class_prompts

cls_prompts = process_class_prompts(generate_chexpert_class_prompts(n=10))
inputs['prompt_inputs'] = cls_prompts

### Run classification

In [ ]:
output = clf(**inputs)
print(output)